Copyright (c) Microsoft Corporation. All rights reserved.
Licensed under the MIT License.

# Optimizing runtime performance on GPT-2 model inference with ONNXRuntime on CPU

In this tutorial, you'll be introduced to how to load a GPT2 model from PyTorch, convert it to ONNX with one step search, and inference it using ONNX Runtime with/without IO Binding. GPT-2 model inference is optimized by compiling one-step beam search into the onnx compute graph, which speeds up the runtime significantly. 

## Prerequisites
If you have Jupyter Notebook, you may directly run this notebook. We will use pip to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/) and other required packages.

Otherwise, you can setup a new environment. First, we install [Anaconda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.8
conda activate cpu_env
conda install jupyter
jupyter notebook
```

The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

In [1]:
# Install PyTorch 1.7.0 and OnnxRuntime 1.7.0 for CPU-only.
import sys
if sys.platform == 'darwin': # Mac
    !{sys.executable} -m pip install --upgrade torch torchvision
else:
    !{sys.executable} -m pip install --upgrade torch==1.9.0+cpu torchvision==0.10.0+cpu torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install onnxruntime==1.8.1
!{sys.executable} -m pip install sentencepiece

# Install other packages used in this notebook.
!{sys.executable} -m pip install transformers==4.8.2
!{sys.executable} -m pip install onnx onnxconverter_common psutil pytz pandas py-cpuinfo py3nvml sympy coloredlogs azureml-core azureml-mlflow mlflow

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
import os

# Create a cache directory to store pretrained model.
cache_dir = os.path.join(".", "cache_models")
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

## Convert GPT2 model from PyTorch to ONNX with one step search ##

We have a script [convert_to_onnx.py](https://github.com/microsoft/onnxruntime/blob/master/onnxruntime/python/tools/transformers/convert_to_onnx.py) that could help you to convert GPT2 with past state to ONNX. 

The script accepts a pretrained model name or path of a checkpoint directory as input, and converts the model to ONNX. It also verifies that the ONNX model could generate same input as the pytorch model. The usage is like 
```
python -m onnxruntime.transformers.convert_to_onnx -m model_name_or_path \ 
--model_class=GPT2LMHeadModel_BeamSearchStep|GPT2LMHeadModel_ConfigurableOneStepSearch \ 
--output gpt2_onestepsearch.onnx -o -p fp32|fp16|int8
```
The -p option can be used to choose the precision: fp32 (float32), fp16 (mixed precision) or int8 (quantization). The -o option will generate optimized model, which is required for fp16 or int8.

Here we use a pretrained model as example:

In [3]:
from onnxruntime.transformers.gpt2_beamsearch_helper import Gpt2BeamSearchHelper, GPT2LMHeadModel_BeamSearchStep
from transformers import AutoConfig
import torch

model_name_or_path = "rinna/japanese-gpt2-medium"
config = AutoConfig.from_pretrained(model_name_or_path, cache_dir=cache_dir)
model = GPT2LMHeadModel_BeamSearchStep.from_pretrained(model_name_or_path, config=config, batch_size=1, beam_size=4, cache_dir=cache_dir)
device = torch.device("cpu")
model.eval().to(device)

print(model.config)

num_attention_heads = model.config.n_head
hidden_size = model.config.n_embd
num_layer = model.config.n_layer

GPT2Config {
  "_name_or_path": "rinna/japanese-gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "batch_size": 1,
  "beam_size": 4,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": 4096,
  "n_layer": 24,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.8.2",
  "use_cache": true,
  "vocab_size": 32000
}



In [4]:
onnx_model_path = "rinna_gpt2_beam_step_search.onnx"

if not os.path.exists(onnx_model_path):
    Gpt2BeamSearchHelper.export_onnx(model, device, onnx_model_path) # add parameter use_external_data_format=True when model size > 2 GB
else:
    print("GPT-2 ONNX model exists.")


from onnxruntime.transformers.gpt2_helper import Gpt2Helper, MyGPT2LMHeadModel
from onnxruntime.transformers.quantize_helper import QuantizeHelper

optimized_fp32_model_path = "rinna_gpt2_beam_step_search_optimized_fp32.onnx"
quantized_int8_model_path = "rinna_gpt2_beam_step_search_optimized_gpt2_int8.onnx"

if not os.path.exists(optimized_fp32_model_path):
    Gpt2Helper.optimize_onnx(onnx_model_path, optimized_fp32_model_path, False, model.config.num_attention_heads, model.config.hidden_size)
else:
    print("Optimized GPT-2 FP32 ONNX model exists.")

if not os.path.exists(quantized_int8_model_path):   
    QuantizeHelper.quantize_onnx_model(optimized_fp32_model_path, quantized_int8_model_path)
else:
    print("Quantized GPT-2 Int8 ONNX model exists.")


GPT-2 ONNX model exists.
Optimized GPT-2 FP32 ONNX model exists.
Quantized GPT-2 Int8 ONNX model exists.


## ONNX Runtime Inference ##

We can use ONNX Runtime to inference. The inputs are dictionary with name and numpy array as value, and the output is list of numpy array. Note that both input and output are in CPU. When you run the inference in GPU, it will involve data copy between CPU and GPU for input and output.

Let's create an inference session for ONNX Runtime given the exported ONNX model, and see the output.

In [5]:
import onnxruntime
import numpy
from transformers import T5Tokenizer

EXAMPLE_Text = ['私はりんなです。']

def get_tokenizer(model_name_or_path, cache_dir):
    tokenizer = T5Tokenizer.from_pretrained(model_name_or_path, cache_dir=cache_dir)
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.do_lower_case = True
    #okenizer.add_special_tokens({'pad_token': '[PAD]'})
    return tokenizer

def get_example_inputs(prompt_text=EXAMPLE_Text):    
    tokenizer = get_tokenizer('rinna/japanese-gpt2-medium', cache_dir)
    encodings_dict = tokenizer.batch_encode_plus(prompt_text, padding=True)

    input_ids = torch.tensor(encodings_dict['input_ids'], dtype=torch.int64)
    attention_mask = torch.tensor(encodings_dict['attention_mask'], dtype=torch.float32)
    position_ids = (attention_mask.long().cumsum(-1) - 1)
    position_ids.masked_fill_(position_ids < 0, 0)

    #Empty Past State for generating first word
    empty_past = []
    batch_size = input_ids.size(0)
    sequence_length = input_ids.size(1)
    past_shape = [2, batch_size, num_attention_heads, 0, hidden_size // num_attention_heads]
    for i in range(num_layer):
        empty_past.append(torch.empty(past_shape).type(torch.float32).to(device))
       
    return input_ids, attention_mask, position_ids, empty_past

input_ids, attention_mask, position_ids, empty_past = get_example_inputs()
beam_select_idx = torch.zeros([1, input_ids.shape[0]]).long()
input_log_probs = torch.zeros([input_ids.shape[0], 1])
input_unfinished_sents = torch.ones([input_ids.shape[0], 1], dtype=torch.bool)
prev_step_scores = torch.zeros([input_ids.shape[0], 1])

session = onnxruntime.InferenceSession(onnx_model_path)
ort_inputs = {
              'input_ids': numpy.ascontiguousarray(input_ids.cpu().numpy()),
              'attention_mask' : numpy.ascontiguousarray(attention_mask.cpu().numpy()),
              'position_ids': numpy.ascontiguousarray(position_ids.cpu().numpy()),
              'beam_select_idx': numpy.ascontiguousarray(beam_select_idx.cpu().numpy()),
              'input_log_probs': numpy.ascontiguousarray(input_log_probs.cpu().numpy()),
              'input_unfinished_sents': numpy.ascontiguousarray(input_unfinished_sents.cpu().numpy()),
              'prev_step_results': numpy.ascontiguousarray(input_ids.cpu().numpy()),
              'prev_step_scores': numpy.ascontiguousarray(prev_step_scores.cpu().numpy()),
             }
for i, past_i in enumerate(empty_past):
    ort_inputs[f'past_{i}'] = numpy.ascontiguousarray(past_i.cpu().numpy())
print(ort_inputs)
ort_outputs = session.run(None, ort_inputs)
print(ort_outputs)

{'input_ids': array([[   9, 6057, 7670,   57, 2767,    8,    2]]), 'attention_mask': array([[1., 1., 1., 1., 1., 1., 1.]], dtype=float32), 'position_ids': array([[0, 1, 2, 3, 4, 5, 6]]), 'beam_select_idx': array([[0]]), 'input_log_probs': array([[0.]], dtype=float32), 'input_unfinished_sents': array([[ True]]), 'prev_step_results': array([[   9, 6057, 7670,   57, 2767,    8,    2]]), 'prev_step_scores': array([[0.]], dtype=float32), 'past_0': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_1': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_2': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_3': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_4': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_5': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_6': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_7': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_8': array([], shape=(2, 1, 16, 0, 64), dtype=float32), 'past_9': arra

## ONNX Runtime Inference with IO Binding ##

To avoid data copy for input and output, ONNX Runtime also supports IO Binding. User could provide some buffer for input and outputs. For GPU inference, the buffer can be in GPU to reduce memory copy between CPU and GPU. This is helpful for high performance inference in GPU. For GPT-2, IO Binding might help the performance when batch size or (past) sequence length is large.

In [6]:
def inference_with_io_binding(session, config, input_ids, position_ids, attention_mask, past, beam_select_idx, input_log_probs, input_unfinished_sents, prev_step_results, prev_step_scores, step, context_len):
    output_shapes = Gpt2BeamSearchHelper.get_output_shapes(batch_size=1,
                                                           context_len=context_len,
                                                           past_sequence_length=past[0].size(3),
                                                           sequence_length=input_ids.size(1),
                                                           beam_size=4,
                                                           step=step,
                                                           config=config,
                                                           model_class="GPT2LMHeadModel_BeamSearchStep")
    output_buffers = Gpt2BeamSearchHelper.get_output_buffers(output_shapes, device)

    io_binding = Gpt2BeamSearchHelper.prepare_io_binding(session, input_ids, position_ids, attention_mask, past, output_buffers, output_shapes, beam_select_idx, input_log_probs, input_unfinished_sents, prev_step_results, prev_step_scores)
    session.run_with_iobinding(io_binding)

    outputs = Gpt2BeamSearchHelper.get_outputs_from_io_binding_buffer(session, output_buffers, output_shapes, return_numpy=False)
    return outputs

We can see that the result is exactly same with/without IO Binding:

In [7]:
input_ids, attention_mask, position_ids, empty_past = get_example_inputs()
beam_select_idx = torch.zeros([1, input_ids.shape[0]]).long()
input_log_probs = torch.zeros([input_ids.shape[0], 1])
input_unfinished_sents = torch.ones([input_ids.shape[0], 1], dtype=torch.bool)
prev_step_scores = torch.zeros([input_ids.shape[0], 1])
outputs = inference_with_io_binding(session, config, input_ids, position_ids, attention_mask, empty_past, beam_select_idx, input_log_probs, input_unfinished_sents, input_ids, prev_step_scores, 0, input_ids.shape[-1])
assert torch.eq(outputs[-2], torch.from_numpy(ort_outputs[-2])).all()
print("IO Binding result is good")

IO Binding result is good


## Batch Text Generation ##

Here is an example for text generation using ONNX Runtime with/without IO Binding.

In [8]:
def update(output, step, batch_size, beam_size, context_length, prev_attention_mask, device):
    """
    Update the inputs for next inference.
    """
    last_state = (torch.from_numpy(output[0]).to(device)
                        if isinstance(output[0], numpy.ndarray) else output[0].clone().detach().cpu())

    input_ids = last_state.view(batch_size * beam_size, -1).to(device)

    input_unfinished_sents_id = -3
    prev_step_results = (torch.from_numpy(output[-2]).to(device) if isinstance(output[-2], numpy.ndarray)
                                else output[-2].clone().detach().to(device))
    position_ids = (torch.tensor([context_length + step - 1
                                        ]).unsqueeze(0).repeat(batch_size * beam_size, 1).to(device))

    if prev_attention_mask.shape[0] != (batch_size * beam_size):
        prev_attention_mask = prev_attention_mask.repeat(batch_size * beam_size, 1)
    attention_mask = torch.cat(
        [
            prev_attention_mask,
            torch.ones([batch_size * beam_size, 1]).type_as(prev_attention_mask),
        ],
        1,
    ).to(device)

    beam_select_idx = (torch.from_numpy(output[input_unfinished_sents_id - 2]).to(device) if isinstance(
        output[input_unfinished_sents_id - 2], numpy.ndarray) else output[input_unfinished_sents_id - 2].clone().detach().to(device))
    input_log_probs = (torch.from_numpy(output[input_unfinished_sents_id - 1]).to(device) if isinstance(
        output[input_unfinished_sents_id - 1], numpy.ndarray) else output[input_unfinished_sents_id - 1].clone().detach().to(device))
    input_unfinished_sents = (torch.from_numpy(output[input_unfinished_sents_id]).to(device) if isinstance(
        output[input_unfinished_sents_id], numpy.ndarray) else
                                    output[input_unfinished_sents_id].clone().detach().to(device))
    prev_step_scores = (torch.from_numpy(output[-1]).to(device)
                                if isinstance(output[-1], numpy.ndarray) else output[-1].clone().detach().to(device))

    past = []
    if isinstance(output[1], tuple):  # past in torch output is tuple
        past = list(output[1])
    else:
        for i in range(model.config.n_layer):
            past_i = (torch.from_numpy(output[i + 1])
                        if isinstance(output[i + 1], numpy.ndarray) else output[i + 1].clone().detach())
            past.append(past_i.to(device)) 

    inputs = {
        'input_ids': input_ids,
        'attention_mask' : attention_mask,
        'position_ids': position_ids,
        'beam_select_idx': beam_select_idx,
        'input_log_probs': input_log_probs,
        'input_unfinished_sents': input_unfinished_sents,
        'prev_step_results': prev_step_results,
        'prev_step_scores': prev_step_scores,
    }
    ort_inputs = {
        'input_ids': numpy.ascontiguousarray(input_ids.cpu().numpy()),
        'attention_mask' : numpy.ascontiguousarray(attention_mask.cpu().numpy()),
        'position_ids': numpy.ascontiguousarray(position_ids.cpu().numpy()),
        'beam_select_idx': numpy.ascontiguousarray(beam_select_idx.cpu().numpy()),
        'input_log_probs': numpy.ascontiguousarray(input_log_probs.cpu().numpy()),
        'input_unfinished_sents': numpy.ascontiguousarray(input_unfinished_sents.cpu().numpy()),
        'prev_step_results': numpy.ascontiguousarray(prev_step_results.cpu().numpy()),
        'prev_step_scores': numpy.ascontiguousarray(prev_step_scores.cpu().numpy()),
    }
    for i, past_i in enumerate(past):
        ort_inputs[f'past_{i}'] = numpy.ascontiguousarray(past_i.cpu().numpy())
    
    return inputs, ort_inputs, past

def test_generation(tokenizer, input_text, use_onnxruntime_io, ort_session = None, num_tokens_to_produce = 30):
    print("Text generation using", "OnnxRuntime with IO binding" if use_onnxruntime_io else "OnnxRuntime", "...")    
    input_ids, attention_mask, position_ids, past = get_example_inputs(input_text)
    beam_select_idx = torch.zeros([1, input_ids.shape[0]]).long()
    input_log_probs = torch.zeros([input_ids.shape[0], 1])
    input_unfinished_sents = torch.ones([input_ids.shape[0], 1], dtype=torch.bool)
    prev_step_scores = torch.zeros([input_ids.shape[0], 1])
    inputs = {
        'input_ids': input_ids,
        'attention_mask' : attention_mask,
        'position_ids': position_ids,
        'beam_select_idx': beam_select_idx,
        'input_log_probs': input_log_probs,
        'input_unfinished_sents': input_unfinished_sents,
        'prev_step_results': input_ids,
        'prev_step_scores': prev_step_scores,
    }
    ort_inputs = {
        'input_ids': numpy.ascontiguousarray(input_ids.cpu().numpy()),
        'attention_mask' : numpy.ascontiguousarray(attention_mask.cpu().numpy()),
        'position_ids': numpy.ascontiguousarray(position_ids.cpu().numpy()),
        'beam_select_idx': numpy.ascontiguousarray(beam_select_idx.cpu().numpy()),
        'input_log_probs': numpy.ascontiguousarray(input_log_probs.cpu().numpy()),
        'input_unfinished_sents': numpy.ascontiguousarray(input_unfinished_sents.cpu().numpy()),
        'prev_step_results': numpy.ascontiguousarray(input_ids.cpu().numpy()),
        'prev_step_scores': numpy.ascontiguousarray(prev_step_scores.cpu().numpy()),
    }
    for i, past_i in enumerate(past):
        ort_inputs[f'past_{i}'] = numpy.ascontiguousarray(past_i.cpu().numpy())
    batch_size = input_ids.size(0)
    beam_size = 4
    context_length = input_ids.size(-1)

    for step in range(num_tokens_to_produce):
        if use_onnxruntime_io:
            outputs = inference_with_io_binding(ort_session, config, inputs['input_ids'], inputs['position_ids'], inputs['attention_mask'], past, inputs['beam_select_idx'], inputs['input_log_probs'], inputs['input_unfinished_sents'], inputs['prev_step_results'], inputs['prev_step_scores'], step, context_length)
        else:
            outputs = ort_session.run(None, ort_inputs) 
        inputs, ort_inputs, past = update(outputs, step, batch_size, beam_size, context_length, inputs['attention_mask'], device)

        if not inputs['input_unfinished_sents'].any():
            break

    print("------------")
    print(tokenizer.decode(inputs['prev_step_results'][0], skip_special_tokens=True))

In [9]:
import time
tokenizer = get_tokenizer(model_name_or_path, cache_dir)
input_text = EXAMPLE_Text

In [10]:
start = time.time()
test_generation(tokenizer, input_text, use_onnxruntime_io=False, ort_session=session)    
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

Text generation using OnnxRuntime ...
------------
私はりんなです。よろしくお願いします。 はじめまして。 プロフィールをご覧いただきありがとうございます。 はじめまして。
elapsed_time:4.864267826080322[sec]


Next, we use ONNX Runtime with IO binding to run again and we can see that the result is exactly same.

In [11]:
start = time.time()
test_generation(tokenizer, input_text, use_onnxruntime_io=True, ort_session=session)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

Text generation using OnnxRuntime with IO binding ...
------------
私はりんなです。よろしくお願いします。 はじめまして。 プロフィールをご覧いただきありがとうございます。 はじめまして。
elapsed_time:5.5472471714019775[sec]


Quantatize GPT-2

In [12]:
session_int8 = onnxruntime.InferenceSession(quantized_int8_model_path)

start = time.time()
test_generation(tokenizer, input_text, use_onnxruntime_io=False, ort_session=session_int8)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

Text generation using OnnxRuntime ...
------------
私はりんなです。よろしくお願いします。 はじめまして。 プロフィールをご覧いただきありがとうございます。 はじめまして。
elapsed_time:4.197372913360596[sec]


# mlflow models

In [13]:
import os
from azureml.core import Workspace
import mlflow

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
experiment_name = 'experiment_with_mlflow'
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    remote_quantized_int8_model_path = 'onnx/'+ quantized_int8_model_path
    mlflow.log_artifact(quantized_int8_model_path, 'onnx/'+ quantized_int8_model_path)
    model_uri = "runs:/{}/".format(run.info.run_id) + remote_quantized_int8_model_path # run_id を持っておけば with mlflow.start_run() as run は不要です
    mlflow.register_model(model_uri, 'rinna-GPT2-quantized_int8-model')


Successfully registered model 'rinna-GPT2-model'.
2021/08/04 09:12:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: rinna-GPT2-model, version 1
Created version '1' of model 'rinna-GPT2-model'.
